In [1]:
import os
import pandas as pd
from pathlib import Path

In [2]:
home = '/home/sshankar'
ibc_folder = 'ibc'

In [3]:
# The sessions.csv file contains information on which task 
# was acquired in which session
# Use it to also get the list of subjects and sessions

sessfile = os.path.join(home, ibc_folder, 'public_analysis_code/ibc_data/sessions.csv')
sess_df = pd.read_csv(sessfile)
subjects = sess_df.subject
sessions = sess_df.columns[1:]

In [29]:
# What is the task of interest?
task = 'clips'

In [5]:
subjects

0     sub-01
1     sub-02
2     sub-04
3     sub-05
4     sub-06
5     sub-07
6     sub-08
7     sub-09
8     sub-11
9     sub-12
10    sub-13
11    sub-14
12    sub-15
Name: subject, dtype: object

In [15]:
sessions

Index(['ses-00', 'ses-01', 'ses-02', 'ses-03', 'ses-04', 'ses-05', 'ses-06',
       'ses-07', 'ses-08', 'ses-09', 'ses-10', 'ses-11', 'ses-12', 'ses-13',
       'ses-14', 'ses-15', 'ses-16', 'ses-17', 'ses-18', 'ses-19', 'ses-20',
       'ses-21', 'ses-22', 'ses-23', 'ses-24', 'ses-25', 'ses-26', 'ses-27',
       'ses-28', 'ses-29', 'ses-30'],
      dtype='object')

In [30]:
# For each subject, find which sessions contain the task of interest
task_sess = []

for i in range(len(subjects)):
    if subjects[i] in subs:
        ser = sess_df.iloc[i,:]
        ids = ser.str.contains(task)==True
        sess = ser.loc[ids].keys().tolist()
        task_sess.append(sess)

In [9]:
ser

subject           sub-15
ses-00         screening
ses-01             archi
ses-02              hcp1
ses-03              hcp2
ses-04     rsvp-language
ses-05               tom
ses-06       preferences
ses-07            clips1
ses-08            clips2
ses-09            clips3
ses-10       enumeration
ses-11            clips4
ses-12             anat1
ses-13             anat2
ses-14              mtt1
ses-15              mtt2
ses-16          raiders1
ses-17             anat1
ses-18             lyon1
ses-19              self
ses-20             lyon2
ses-21            audio1
ses-22            audio2
ses-23         stanford1
ses-24               NaN
ses-25               NaN
ses-26               NaN
ses-27               NaN
ses-28               NaN
ses-29               NaN
ses-30               NaN
Name: 12, dtype: object

In [31]:
task_sess

[['ses-06', 'ses-08', 'ses-09', 'ses-11'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-09'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-09'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-06', 'ses-07', 'ses-08', 'ses-10'],
 ['ses-04', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-07', 'ses-08', 'ses-09', 'ses-10'],
 ['ses-07', 'ses-08', 'ses-09', 'ses-11']]

In [17]:
task_sess

[['ses-06', 'ses-08', 'ses-09', 'ses-11'],
 ['ses-07', 'ses-08', 'ses-09'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-09'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-09'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-06', 'ses-07', 'ses-08', 'ses-10'],
 ['ses-04', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-05', 'ses-06', 'ses-07', 'ses-08'],
 ['ses-07', 'ses-08', 'ses-09', 'ses-10'],
 ['ses-07', 'ses-08', 'ses-09', 'ses-11']]

In [23]:
subs = ['sub-01', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 
        'sub-09', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15']
# raiders1 = [16, 13, 13, 13, 14, 14, 14, 14, 13, 14]
# raiders2 = [17, 14, 14, 14, 15, 15, 15, 15, 14, 15]

In [134]:
data_dir_local = os.path.join(home, task, '3mm')
data_dir_drago = 'sshankar@drago:/storage/store/data/ibc/3mm/'

In [135]:
# Create directories for subjects and sessions
for i, sub in enumerate(subjects):
    for ses in task_sess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        if not os.path.isdir(ses_path):
            os.makedirs(ses_path)

In [137]:
for i, sub in enumerate(subjects):
    for ses in task_sess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        os.chdir(ses_path)
        remotefile = os.path.join(data_dir_drago, sub, ses, 'func/wrd*')
        os.system('scp %s .' % (remotefile) )